# Playground

In [88]:
import math
import numpy as np

class LunakDense:
    def __init__(self, units, activation, input_dim, init, use_bias=False, seed=None):
        self.units = units
        self.input_dim = input_dim
        
        if activation == 'sigmoid':
            self.activation_function = self.sigmoid
        else:
            print('Activation function not supported')
        
        np.random.seed(seed)
        
        if init == 'uniform':
            self.weight_matrix = np.random.uniform(-0.05, 0.05, size=(self.units, input_dim)) 
            print(self.weight_matrix)
        elif init == 'random':
            self.weight_matrix = np.random.random(size=(self.units, input_dim))
        else:
            print('Init function not supported')
        
        self.delta_weight_matrix_before = np.zeros((self.units, input_dim))
        self.delta_weight_matrix = np.zeros((self.units, input_dim))
        
        self.use_bias = use_bias
        if self.use_bias:
            bias = np.zeros((units, 1))
            self.weight_matrix = np.hstack((self.weight_matrix, bias))
            self.delta_weight_matrix_before = np.hstack((self.delta_weight_matrix_before, np.zeros((units, 1))))
            self.delta_weight_matrix = np.hstack((self.delta_weight_matrix, np.zeros((units, 1))))
            
    def calculate_sigma(self, input_list):
        if self.use_bias:
            input_list = np.append(input_list, 1)
        
        result_list = np.array([])
        for weight_neuron in self.weight_matrix:
            result_list = np.append(result_list, np.dot(weight_neuron, input_list))
        return np.array(result_list)
    
    def calculate_output(self, input_list):
        output_list = np.array([])
        for sigma_neuron in self.calculate_sigma(input_list):
            output_list = np.append(output_list, self.activation_function(sigma_neuron))
        self.output_list = output_list
        return output_list.copy()
    
    def calculate_local_gradient_output_layer(self, target_list):
        """
        Use this if the layer is output layer
        """
        result_list = np.array([])
        for index, output in enumerate(self.output_list):
            local_gradient = output * (1 - output) * (target_list[index] - output)
            result_list = np.append(result_list, local_gradient)  
        self.local_gradient = result_list
        return result_list.copy()
    
    def calculate_local_gradient_hidden_layer(self, local_gradient_output_list, output_layer_weight_matrix):
        """
        Use this if the layer is hidden layer
        """
        result_list = np.array([])
        for index, output in enumerate(self.output_list):
            sigma_local_gradient_output = 0
            for unit_number, local_gradient in enumerate(local_gradient_output_list):
                sigma_local_gradient_output += output_layer_weight_matrix[unit_number][index] * local_gradient
            error_hidden = output * (1 - output) * sigma_local_gradient_output
            result_list = np.append(result_list, error_hidden)
        self.local_gradient = result_list
        return result_list.copy()
    
    def update_delta_weight(self, lr, input_list, momentum=None):
        """
        Function to update delta weight
        """
        if self.use_bias:
            input_list = np.append(input_list, 1)
        if momentum == None:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * self.local_gradient[j] * input_list[i]
#                     new_weight = source + delta_weight
#                     self.weight_matrix[j][i] = new_weight
                    self.delta_weight_matrix[j][i] = delta_weight.copy()
        else:
            for j, unit in enumerate(self.weight_matrix): #j  
                for i, source in enumerate(unit): #i
                    delta_weight = lr * self.local_gradient[j] * input_list[i] + momentum * self.delta_weight_matrix_before[j][i]
                    
                    # Update Delta Weight
                    self.delta_weight_matrix_before[j][i] = delta_weight.copy()
                    
#                     new_weight = source + delta_weight
#                     self.weight_matrix[j][i] = new_weight
            
            # Copy Last Update of Weight Matrix Before (Equal to Last Weight Matrix)
            for j, unit in enumerate(self.delta_weight_matrix_before):
                for i, source in enumerate(unit):
                    self.delta_weight_matrix[j][i] = self.delta_weight_matrix_before[j][i].copy()
            
    def update_weight(self):
        """
        Function to update weight
        """
        for j, unit in enumerate(self.delta_weight_matrix_before):
            for i, source in enumerate(unit):
                self.weight_matrix[j][i] += self.delta_weight_matrix[j][i]
    
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

In [352]:
np.random.seed(5)
layer = LunakDense(3, 'sigmoid', 2, 'uniform', True)
layer.calculate_sigma([3, 2])

array([2.40744413, 2.45737928, 2.68872129])

In [353]:
layer.calculate_output([3, 2])

array([0.9173932 , 0.92109941, 0.93635782])

# Test Feedforward

In [354]:
layer_hidden = LunakDense(2, 'sigmoid', 2, 'uniform', True)
layer_hidden.weight_matrix = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])

layer_hidden.calculate_output([0.1, 0.9])

array([0.54239794, 0.58904043])

# Test Calculate Local Gradient Hidden Layer

In [355]:
layer_hidden_output = LunakDense(2, 'sigmoid', 2, 'uniform', True)
layer_hidden_output.output_list = [0.542, 0.589]
layer_hidden_output.calculate_local_gradient_hidden_layer([0.0663], [[0.2, 0.3]])

array([0.00329161, 0.00481495])

# Test Calculate Local Gradient Output Layer

In [356]:
layer_error_output = LunakDense(1, 'sigmoid', 2, 'uniform', True)
layer_error_output.output_list = [0.619]
layer_error_output.calculate_local_gradient_output_layer([0.9])

array([0.06627076])

# Test Update Weight Momentum 1

In [357]:
layer_test_update_weight = LunakDense(1, 'sigmoid', 2, 'uniform', True)
layer_test_update_weight.weight_matrix = np.array([[0.2, 0.3, 0.2]])

layer_test_update_weight.delta_weight_matrix_before = np.array([[0.2, 0.3, 0.2]])
print(layer_test_update_weight.weight_matrix)
print(layer_test_update_weight.delta_weight_matrix_before)

layer_test_update_weight.local_gradient = [0.0663]
layer_test_update_weight.update_delta_weight(0.25, [0.542, 0.589], 0.0001)
layer_test_update_weight.update_weight()
print(layer_test_update_weight.delta_weight_matrix_before)
print(layer_test_update_weight.weight_matrix)

[[0.2 0.3 0.2]]
[[0.2 0.3 0.2]]
[[0.00900365 0.00979267 0.016595  ]]
[[0.20900365 0.30979267 0.216595  ]]


# Test Update Weight Momentum 2

In [359]:
layer_test_update_weight2 = LunakDense(2, 'sigmoid', 2, 'uniform', True)
layer_test_update_weight2.weight_matrix = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])

layer_test_update_weight2.delta_weight_matrix_before = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])
print(layer_test_update_weight2.weight_matrix)
print(layer_test_update_weight2.delta_weight_matrix_before)

layer_test_update_weight2.local_gradient = [0.0033, 0.0049]
layer_test_update_weight2.update_delta_weight(0.25, [0.1, 0.9], 0.0001)
layer_test_update_weight2.update_weight()
print(layer_test_update_weight2.delta_weight_matrix_before)
print(layer_test_update_weight2.weight_matrix)

[[-0.2  0.1  0.1]
 [-0.1  0.3  0.1]]
[[-0.2  0.1  0.1]
 [-0.1  0.3  0.1]]
[[6.2500e-05 7.5250e-04 8.3500e-04]
 [1.1250e-04 1.1325e-03 1.2350e-03]]
[[-0.1999375  0.1007525  0.100835 ]
 [-0.0998875  0.3011325  0.101235 ]]


# Test Update Weight Without Momentum TODO

In [247]:
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

# Model Class

In [268]:
class LunakArtificialNeuralNetwork:
    def __init__(self, loss='root_mean_squared', optimizer='sgd'):
        assert loss == 'root_mean_squared', 'loss function not supported'
        assert optimizer == 'sgd', 'optimizer not supported'
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)
        
    def feed_forward(self, X_instance):
        # Calculate output with the first hidden layer
        output_list = self.layers[0].calculate_output(X_instance)
        # Calculate output with the second until the last layer
        for layer in self.layers[1:]:
            next_output_list = layer.calculate_output(output_list)
            output_list = next_output_list
        return output_list.copy()
            
    def backpropagation(self, y_instance):
        # Calculate local gradient for output layer
        next_local_gradient_list = self.layers[-1].calculate_local_gradient_output_layer([y_instance])
        next_layer_weight_matrix = self.layers[-1].weight_matrix

        # Calculate local gradient for hidden layer(s)
        for layer_idx, layer in enumerate(reversed(self.layers[0:-1])):
            next_local_gradient_list = layer.calculate_local_gradient_hidden_layer(next_local_gradient_list, next_layer_weight_matrix)
            next_layer_weight_matrix = layer.weight_matrix
            
    def calculate_delta_weight(self, X_instance, lr, momentum):
        # Update delta weight for first hidden layer
        self.layers[0].update_delta_weight(lr, X_instance, momentum)
        
        # Update delta weight for other layers
        for layer_idx, layer in enumerate(self.layers[1:]):
            layer.update_delta_weight(lr, self.layers[layer_idx].output_list, momentum)
    
    def fit(self, X, y, epochs, lr, momentum=None, batch_size=None, val_data=None, val_size=0):
        assert X.shape[1] == self.layers[0].input_dim, 'Input dimension must be same with the column'
        self.classes_ = np.unique(y)
        
        if batch_size == None:
            batch_size = len(X)
            
        if val_data is None:
            val_size = 0.1
            X, X_val, y, y_val = train_test_split(X, y, test_size=val_size)
        else:
            X_val = val_data[0]
            y_val = val_data[1]
        
        if val_data is not None and val_size != 0:
            print('Validation data will be used instead of val_size.')
            
        for epoch in range(epochs):
            print('epoch', epoch)
            # SGD Batch / Mini Batch
            delta = len(X) // batch_size
            for start in range(0, len(X), delta):
                X_batch = X[start:start+delta]
                y_batch = y[start:start+delta]
                
                for idx, X_instance in enumerate(X_batch):
                    self.feed_forward(X_instance)
                    self.backpropagation(y_batch[idx][0])
                    self.calculate_delta_weight(X_instance, lr, momentum)

                for layer in self.layers:
                    layer.update_weight()
                    
            pred = self.predict(X)
            pred_val = self.predict(X_val)
            
            print('acc:', self.calculate_accuracy(y, pred))
            print('val_acc:', self.calculate_accuracy(y_val, pred_val))
            
            # Loss
            print('loss:', mean_squared_error(y, pred))
            print('val_loss:', mean_squared_error(y_val, pred_val))
            print()
    
    def predict_proba(self, X):
        predictions = []
        for idx, X_instance in enumerate(X):
            X_pred = self.feed_forward(X_instance)
            predictions.append([np.mean(X_pred.copy())])
        return predictions
    
    def predict(self, X):
        predictions = []
        for idx, X_instance in enumerate(X):
            X_pred_proba = self.feed_forward(X_instance)
            X_pred = min(self.classes_, key=lambda pred_class:abs(pred_class - np.mean(X_pred_proba)))
            predictions.append([X_pred.copy()])
        return predictions
    
    def calculate_accuracy(self, y_true, y_pred):
        if len(confusion_matrix(y_true, y_pred).ravel()) > 1:
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        else:
            tp = confusion_matrix(y_true, y_pred).ravel()[0]
            fp = 0
            fn = 0
            tn = 0
        return (tp + tn) / (tp + tn + fp + fn)

In [491]:
model = LunakArtificialNeuralNetwork()

In [492]:
model.add(LunakDense(2, 'sigmoid', 2, 'uniform', True))
model.add(LunakDense(1, 'sigmoid', 2, 'uniform', True))

In [493]:
model.layers[0].weight_matrix = np.array([[-0.2, 0.1, 0.1], [-0.1, 0.3, 0.1]])
model.layers[1].weight_matrix = np.array([[0.2, 0.3, 0.2]])
print(model.layers[0].weight_matrix)

print(model.layers[1].weight_matrix)

[[-0.2  0.1  0.1]
 [-0.1  0.3  0.1]]
[[0.2 0.3 0.2]]


In [494]:
X_train = np.array([
    [0.1, 0.9]
])
y_train = np.array([
    0.9
])

In [495]:
model.fit(X_train, y_train, 1, 0.25, 0.0001)

IndexError: invalid index to scalar variable.

In [367]:
print(model.layers[0].weight_matrix)
print(model.layers[0].local_gradient)

[[-0.19991775  0.10074028  0.10082253]
 [-0.09987967  0.30108299  0.10120332]]
[0.00329012 0.00481328]


In [368]:
print(model.layers[1].weight_matrix)
print(model.layers[1].local_gradient)

[[0.20898739 0.30976024 0.21656973]]
[0.06627891]


In [369]:
list(set([1, 2, 4, 4, 5, 9, 0]))

[0, 1, 2, 4, 5, 9]

# Artificial Neural Network

Agus Gunawan, Holy Lovenia, Felix Limanta

## Data Preparation

In [3]:
from scipy.io.arff import loadarff
import pandas as pd

### Read weather data

In [4]:
raw_data = loadarff('dataset/weather.arff')

In [5]:
data = pd.DataFrame(raw_data[0])

In [6]:
data.head()

,outlook,temperature,humidity,windy,play
0,b'sunny',85.0,85.0,b'FALSE',b'no'
1,b'sunny',80.0,90.0,b'TRUE',b'no'
2,b'overcast',83.0,86.0,b'FALSE',b'yes'
3,b'rainy',70.0,96.0,b'FALSE',b'yes'
4,b'rainy',68.0,80.0,b'FALSE',b'yes'


### Preprocessing

In [7]:
def convert_to_binary_vector(data):
    return pd.get_dummies(data)

In [8]:
for idx, column in enumerate(['outlook', 'windy', 'play']):
    data[column] = data[column].str.decode('utf-8')

In [9]:
bv_outlook = convert_to_binary_vector(data['outlook'])
bv_outlook.head()

,overcast,rainy,sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0


In [10]:
bv_windy = convert_to_binary_vector(data['windy'])
bv_windy.head()

,FALSE,TRUE
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [11]:
preproc_data = data.drop('outlook', 1).drop('windy', 1)

In [12]:
preproc_data['play'] = preproc_data['play'].astype('category')

In [13]:
preprocessed_data = pd.concat([bv_outlook, bv_windy, preproc_data], axis=1)
preprocessed_data.head()

,overcast,rainy,sunny,FALSE,TRUE,temperature,humidity,play
0,0,0,1,1,0,85.0,85.0,no
1,0,0,1,0,1,80.0,90.0,no
2,1,0,0,1,0,83.0,86.0,yes
3,0,1,0,1,0,70.0,96.0,yes
4,0,1,0,1,0,68.0,80.0,yes


In [14]:
y = pd.DataFrame({'play': preprocessed_data['play'].cat.codes})
y.head()

,play
0,0
1,0
2,1
3,1
4,1


In [15]:
X = preprocessed_data.drop('play', 1)

In [16]:
for column in X.columns:
    X[column] = X[column].astype('float')

In [17]:
X.head()

,overcast,rainy,sunny,FALSE,TRUE,temperature,humidity
0,0.0,0.0,1.0,1.0,0.0,85.0,85.0
1,0.0,0.0,1.0,0.0,1.0,80.0,90.0
2,1.0,0.0,0.0,1.0,0.0,83.0,86.0
3,0.0,1.0,0.0,1.0,0.0,70.0,96.0
4,0.0,1.0,0.0,1.0,0.0,68.0,80.0


In [239]:
y = y.astype('float')

In [240]:
X = np.array(X)
y_new = [instance_y[0] for instance_y in np.array(y)]
y = np.array(y)

In [241]:
X

array([[ 0.,  0.,  1.,  1.,  0., 85., 85.],
       [ 0.,  0.,  1.,  0.,  1., 80., 90.],
       [ 1.,  0.,  0.,  1.,  0., 83., 86.],
       [ 0.,  1.,  0.,  1.,  0., 70., 96.],
       [ 0.,  1.,  0.,  1.,  0., 68., 80.],
       [ 0.,  1.,  0.,  0.,  1., 65., 70.],
       [ 1.,  0.,  0.,  0.,  1., 64., 65.],
       [ 0.,  0.,  1.,  1.,  0., 72., 95.],
       [ 0.,  0.,  1.,  1.,  0., 69., 70.],
       [ 0.,  1.,  0.,  1.,  0., 75., 80.],
       [ 0.,  0.,  1.,  0.,  1., 75., 70.],
       [ 1.,  0.,  0.,  0.,  1., 72., 90.],
       [ 1.,  0.,  0.,  1.,  0., 81., 75.],
       [ 0.,  1.,  0.,  0.,  1., 71., 91.]])

In [242]:
y

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]])

## LunakANN

In [264]:
lunak_ann = LunakArtificialNeuralNetwork()

In [265]:
lunak_ann.add(LunakDense(2, 'sigmoid', 7, 'uniform', False, seed=5))
lunak_ann.add(LunakDense(1, 'sigmoid', 2, 'uniform', False, seed=5))

[[-0.02780068  0.03707323 -0.02932808  0.04186109 -0.00115888  0.01117439
   0.02659079]
 [ 0.0018418  -0.02031995 -0.03122788 -0.04192587  0.02384403 -0.00586908
  -0.03416901]]
[[-0.02780068  0.03707323]]


In [267]:
lunak_ann.fit(X, y, epochs=50, momentum=0.001, lr=0.001, batch_size=10, val_size=0.2)

[[1.]
 [1.]] [[ 0.  1.  0.  1.  0. 75. 80.]
 [ 0.  1.  0.  1.  0. 70. 96.]]
epoch 0
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 1
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 2
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 3
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 4
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 5
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 6
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 7
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 8
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 9
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val_loss: 0.0

epoch 10
acc: 0.5833333333333334
val_acc: 1.0
loss: 0.4166666666666667
val

In [219]:
confusion_matrix([0, 0, 0], [1, 1, 1])

array([[0, 3],
       [0, 0]])

In [220]:
predictions = lunak_ann.predict(X)

In [221]:
predictions

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

## Keras

In [118]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomUniform
import keras
import pandas as pd
import tensorflow as tf

In [81]:
keras_ann = Sequential()

In [123]:
initializer = RandomUniform(minval=-0.05, maxval=0.05, seed=5)

In [83]:
keras_ann.add(Dense(2, activation='sigmoid', input_dim=7, use_bias=False, kernel_initializer=initializer))
keras_ann.add(Dense(1, activation='sigmoid', use_bias=False, kernel_initializer=initializer))

In [84]:
optimizer_ = keras.optimizers.SGD(momentum=0.001, lr=0.001)

In [85]:
keras_ann.compile(loss='mean_squared_error', optimizer=optimizer_, metrics=['accuracy'])

In [86]:
keras_ann.fit(X, y, epochs=50, batch_size=10)

Epoch 1/50
14/14 [==============================] - 0s 5ms/step - loss: 0.2495 - acc: 0.6429
Epoch 2/50
14/14 [==============================] - 0s 128us/step - loss: 0.2495 - acc: 0.6429
Epoch 3/50
14/14 [==============================] - 0s 135us/step - loss: 0.2495 - acc: 0.6429
Epoch 4/50
14/14 [==============================] - 0s 131us/step - loss: 0.2495 - acc: 0.6429
Epoch 5/50
14/14 [==============================] - 0s 125us/step - loss: 0.2495 - acc: 0.6429
Epoch 6/50
14/14 [==============================] - 0s 113us/step - loss: 0.2494 - acc: 0.6429
Epoch 7/50
14/14 [==============================] - 0s 109us/step - loss: 0.2494 - acc: 0.6429
Epoch 8/50
14/14 [==============================] - 0s 108us/step - loss: 0.2494 - acc: 0.6429
Epoch 9/50
14/14 [==============================] - 0s 119us/step - loss: 0.2494 - acc: 0.6429
Epoch 10/50
14/14 [==============================] - 0s 108us/step - loss: 0.2494 - acc: 0.6429
Epoch 11/50
14/14 [==============================] 

In [87]:
keras_ann.predict(X)

array([[0.50374407],
       [0.5038381 ],
       [0.5038099 ],
       [0.5039423 ],
       [0.50372106],
       [0.50357056],
       [0.50346035],
       [0.5038726 ],
       [0.50347817],
       [0.5037249 ],
       [0.5035003 ],
       [0.503881  ],
       [0.50362927],
       [0.50390553]], dtype=float32)